In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
import re

locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

'pt_BR.utf8'

In [ ]:
def str_cleasing(str):
    str = re.sub(r'\n', ' ', str)
    str = re.sub(r'\s+', ' ', str)
    str = re.sub(r'\s+$', ' ', str)
    str = re.sub(r'^\s+', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    return str

In [3]:
class DN:

    def __init__(self, base_url='https://diariodonordeste.verdesmares.com.br/'):
        self.base_url = base_url


    def yesterday_url_str(self):
        ontem = datetime.now() - timedelta(days=1)
        data_formatada = ontem.strftime('%d de %B de %Y')
        data_formatada = data_formatada.replace(data_formatada.split()[2], data_formatada.split()[2].capitalize())
        return data_formatada
    

    def getUrls(self, url, word):
        try:
            browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
            page = requests.get(url, headers = browsers)
            resposta = page.text
            soup = BeautifulSoup(resposta, 'html.parser')
            allHrefs = soup.find_all('a')

            selectedUrls = []
            for a in allHrefs:
                newsUrl = a.get('href')
                if (word in newsUrl) & ('?page=' not in newsUrl):
                    selectedUrls.append(f'https://diariodonordeste.verdesmares.com.br{newsUrl}')
            selectedUrls = list(dict.fromkeys(selectedUrls))
            return selectedUrls
        except:
            return []


    def getNewsData(self, url, only_yesterday=True):
        try:
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
            page = requests.get(url, headers = headers)
            resposta = page.text
            soup = BeautifulSoup(resposta, 'html.parser')

            title = soup.find('h1', class_="text-heading text-heading-4xl m-l-article__heading").get_text()
            title = str_cleasing(title)
            title = re.sub(r'\s+', ' ', title).strip()
            
            timestamp = soup.find('time', class_="m-l-article__date_created").get_text()
            isYesterday = self.yesterday_url_str() in timestamp
            
            content = ""
            contentHtml = soup.find_all('div', class_="m-l-article__content space-y-6 prose")
            for c in contentHtml:
                for p in c.find_all('p'):
                    content += p.get_text()
                    title = str_cleasing(title)
                    content += '\n'
            
            if only_yesterday and not isYesterday:
                return None
        
            return {
                "url": url,
                "title": title,
                "content": content
            }
            
        except:
            return None